# **Black Scholes**
---

#### Import Libraries 

In [1]:
import pandas as pd
import numpy as np 

from pandas_datareader import data as web
from datetime import datetime,date
from scipy.stats import norm
from math import log, sqrt, pi, exp

### Define Variables

In [2]:
stock = 'SPY'
expiration = '12-18-2022'
strike_price = 370

### Import Data

In [3]:
today = datetime.now()
one_yr_ago = today.replace(year=today.year-1)

df = web.DataReader(name=stock, data_source='yahoo', start=one_yr_ago, end=today)
df = df.sort_values(by='Date')
df = df.dropna()

df = df.assign(close_day_before=df.Close.shift(1))
df['returns'] = ((df.Close-df.close_day_before) / df.close_day_before)

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 253 entries, 2021-04-28 to 2022-04-27
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   High              253 non-null    float64
 1   Low               253 non-null    float64
 2   Open              253 non-null    float64
 3   Close             253 non-null    float64
 4   Volume            253 non-null    float64
 5   Adj Close         253 non-null    float64
 6   close_day_before  252 non-null    float64
 7   returns           252 non-null    float64
dtypes: float64(8)
memory usage: 17.8 KB


---
## Collecting Variables
---

In [4]:
# σ = volatility (σ is lowercase)
sigma = np.sqrt(252) * df['returns'].std()

# r = risk-free rate (10-yr U.S. Treasury Yield, can get from ^TNX)
usTY10 = '^TNX'
riskFree = (web.DataReader(name=usTY10, data_source='yahoo', start=today.replace(day=today.day-1), end=today)['Close'].iloc[-1])/100

# s = Current Stock Price (spot price, or last close price )
lastCloseP = df['Close'].iloc[-1]

# t = Time To Maturity 
tMature = (datetime.strptime(expiration, "%m-%d-%Y") - datetime.utcnow()).days / 365


print('{} Black Scholes Variables Are:'.format(stock),end='\n')
print('Volatility: ',sigma,end='\n')
print('Risk-Free Rate: ',riskFree,end='\n')
print('Current Price: ',lastCloseP,end='\n')
print('Time to Maturity: ',tMature,end='\n')
print('\n')
print('Determined via Biased Analysis:',end='\n')
print('Strike Price: ',strike_price,end='\n')

SPY Black Scholes Variables Are:
Volatility:  0.1576001708979281
Risk-Free Rate:  0.02818000078201294
Current Price:  417.2699890136719
Time to Maturity:  0.6383561643835617


Determined via Biased Analysis:
Strike Price:  370


---
## Calculate D1 and D2 Values 
---
![BlackScholesFunction](data/black_scholes_formula.png)

In [5]:
def d1(s,k,t,r,sigma):
    return(log(s/k) + (r+sigma**2/2) * t) / (sigma*sqrt(t))

def d2(s,k,t,r,sigma):
    return d1(s,k,t,r,sigma) - sigma*sqrt(t)

---
## Black Scholes: Call Option
---

In [6]:
def bs_call(s,k,t,r,sigma):
    return s*norm.cdf(d1(s,k,t,r,sigma)) - k*exp(-r*t)*norm.cdf(d2(s,k,t,r,sigma))

In [7]:
callOption = bs_call(lastCloseP, strike_price, tMature, riskFree, sigma)

print('Call Option: {}'.format(callOption))

Call Option: 57.24200706648833


---
## Black Scholes: Put Option
---

In [8]:
def bs_put(s,k,t,r,sigma):
    return k*exp(-r*t) - s+bs_call(s,k,t,r,sigma)

In [9]:
putOption = bs_put(lastCloseP, strike_price, tMature, riskFree, sigma)

print('Put Option: {}'.format(putOption))

Put Option: 3.3756420645468665


--- 
## Implied Volatility
---
- expected future volatility of the stock over the life of the option 
- influenced by supply and demand of the underlying option and the market's expectation of the stock price's direction 
- calculated by solving the *Black Scholes* equation backwards for the (σ)volatility starting with option trading price 
- measure if prices are cheap or expensive 
- **High:** high-priced option premiums
- **Low:** demand for option decreasing, prices decrease 

In [10]:
vol_sigma = 0.001

#### Call Option Implied Volatility 

In [11]:
def call_implied_volatility(price,sigma,s,k,t,r):
    og_sig = sigma
    while sigma < 1:
        implied_p = s * norm.cdf(d1(s,k,t,r,sigma))-k*exp(-r*t) * norm.cdf(d2(s,k,t,r,sigma))
        if price-implied_p < og_sig:
            return sigma 
        sigma += og_sig
    return "Not Found"

In [12]:
call_IV = call_implied_volatility(callOption, vol_sigma, lastCloseP, strike_price, tMature, riskFree)

print('Call Option Implied Volatility: {}%'.format(100*call_IV))

Call Option Implied Volatility: 15.800000000000011%


### Put Option Implied Volatility

In [13]:
def put_implied_volatility(price,sigma,s,k,t,r):
    og_sig = sigma
    while sigma < 1: 
        implied_p = k*exp(-r*t) - s + bs_call(s,k,t,r,sigma)
        if price-implied_p < og_sig: 
            return sigma
        sigma += og_sig
    return "Not Found"

In [14]:
put_IV = put_implied_volatility(callOption, vol_sigma, lastCloseP, strike_price, tMature, riskFree)

print('Put Option Implied Volatility: {}%'.format(100*put_IV))

Put Option Implied Volatility: 66.30000000000005%


---
---
# **[GREEKS](greeks.ipynb)**
---
---
## Greeks: Delta
---
- sensitivity of an option's price changes relative to the changes in underlying asset's prices 

In [15]:
# Call Option: Delta
def call_delta(s,k,t,r,sigma): 
    return norm.cdf(d1(s,k,t,r,sigma))

# Put Option: Delta
def put_delta(s,k,t,r,sigma): 
    return -norm.cdf(-d1(s,k,t,r,sigma))

In [16]:
cDelta = call_delta(lastCloseP, strike_price, tMature, riskFree, sigma)
pDelta = put_delta(lastCloseP, strike_price, tMature, riskFree, sigma)

print('Call Option Delta: {}'.format(cDelta),end='\n')
print('Put Option Delta: {}'.format(pDelta),end='\n')

Call Option Delta: 0.8771081359638118
Put Option Delta: -0.12289186403618824


## Greeks: Gamma
---
- Delta's cange relative to the changes in the price of the underlying asset 

In [17]:
# Call Option: Gamma
def call_gamma(s,k,t,r,sigma): 
    return norm.pdf(d1(s,k,t,r,sigma)) / (s*sigma*sqrt(t))

# Put Option: Gamma
def put_gamma(s,k,t,r,sigma): 
    return norm.pdf(d1(s,k,t,r,sigma)) / (s*sigma*sqrt(t))

In [18]:
cGamma = call_gamma(lastCloseP, strike_price, tMature, riskFree, sigma)
pGamma = put_gamma(lastCloseP, strike_price, tMature, riskFree, sigma)

print('Call Option Gamma: {}'.format(cGamma),end='\n')
print('Put Option Gamma: {}'.format(pGamma),end='\n')

Call Option Gamma: 0.003871534761025364
Put Option Gamma: 0.003871534761025364


## Greeks: Vega
---
- sensitivity of an option price relative to the volatility of the underlying asset 

In [19]:
# Call Option: Vega
def call_vega(s,k,t,r,sigma): 
    return 0.01 * (s * norm.pdf(d1(s,k,t,r,sigma))*sqrt(t))

# Put Option: Gamma
def put_vega(s,k,t,r,sigma): 
    return 0.01 * (s * norm.pdf(d1(s,k,t,r,sigma))*sqrt(t))

In [20]:
cVega = call_vega(lastCloseP, strike_price, tMature, riskFree, sigma)
pVega = put_vega(lastCloseP, strike_price, tMature, riskFree, sigma)

print('Call Option Vega: {}'.format(cVega),end='\n')
print('Put Option Vega: {}'.format(pVega),end='\n')

Call Option Vega: 0.6781678612296412
Put Option Vega: 0.6781678612296412


## Greeks: Theta
---
- sensitivity of option price relative to the option's time to maturity 

In [21]:
# Call Option: Theta
def call_theta(s,k,t,r,sigma): 
    return 0.01 * (-(s * norm.pdf(d1(s,k,t,r,sigma))*sigma)/(2*sqrt(t)) - r*k*exp(-r*t)*norm.cdf(d2(s,k,t,r,sigma)))

# Put Option: Theta
def put_theta(s,k,t,r,sigma): 
    return 0.01 * (-(s * norm.pdf(d1(s,k,t,r,sigma))*sigma)/(2*sqrt(t)) + r*k*exp(-r*t)*norm.cdf(-d2(s,k,t,r,sigma)))

In [22]:
cTheta = call_theta(lastCloseP, strike_price, tMature, riskFree, sigma)
pTheta = put_theta(lastCloseP, strike_price, tMature, riskFree, sigma)

print('Call Option Theta: {}'.format(cTheta),end='\n')
print('Put Option Theta: {}'.format(pTheta),end='\n')

Call Option Theta: -0.17071996973152526
Put Option Theta: -0.06831282564315626


## Greeks: Rho
---
- sensitivity of the option price relative to interest rates

In [23]:
# Call Option: Rho
def call_rho(s,k,t,r,sigma): 
    return 0.01 * (k*t*exp(-r*t) * norm.cdf(d2(s,k,t,r,sigma)))

# Put Option: Rho
def put_rho(s,k,t,r,sigma): 
    return 0.01 * (-k*t*exp(-r*t) * norm.cdf(-d2(s,k,t,r,sigma)))

In [24]:
cRho = call_rho(lastCloseP, strike_price, tMature, riskFree, sigma)
pRho = put_rho(lastCloseP, strike_price, tMature, riskFree, sigma)

print('Call Option Rho: {}'.format(cRho),end='\n')
print('Put Option Rho: {}'.format(pRho),end='\n')

Call Option Rho: 1.9709176049174668
Put Option Rho: -0.3488918305546755
